In [23]:
import os
import json
import gzip
import nltk
import seaborn as sns
import pandas as pd
from urllib.request import urlopen
from collections import defaultdict
from sklearn import linear_model
from nltk.corpus import stopwords

# import working with data libraries
import pandas as pd
import numpy as np
import string

# import seaborn
import seaborn as sns

# import matplotlib
import matplotlib.pyplot as plt # Typical way of import MPL
import matplotlib as mpl # This line is used less frequently

In [2]:
### load the review data

review_data = []
with gzip.open('Software.json.gz') as f:
    for l in f:
        review_data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of reviews
print(len(review_data))

# first row of the list
print(review_data[10])

459436
{'overall': 2.0, 'vote': '3', 'verified': True, 'reviewTime': '05 3, 2009', 'reviewerID': 'A1CUJ6ZT82H7LS', 'asin': '0132147556', 'style': {'Format:': ' Paperback'}, 'reviewerName': 'Brian', 'reviewText': "Maybe it's just me (I have no marketing background but desperately want to learn for my start-up) but I cannot get hardly anything out of this text. I have tried very hard to tread through the writing and learn something useful but chapter after chapter seems to be the exact same thing... overly wordy, rambling & unnecessarily academic writing with no comprehensible message behind it, no strategies I can apply to my own business and no organization that I can make sense of. Feels almost like I'm reading a paper a college student BSed their way through with a whole bunch of long words and cryptic sentences in order to sound impressive. What a rip-off. I paid $70 for this and it has been the most useless book I've paid for in my research process (and I've read 16 other books thu

In [3]:
# convert list into pandas dataframe

all_products = {set}
df_review = pd.DataFrame.from_dict(review_data)

#filter out the non-verified reviews, and leave only the columns we want
# df_review = df_review[df_review['verified']==True][["overall", "reviewTime", "asin", "reviewText", "summary"]]
df_review = df_review[["overall", "reviewTime", "asin", "reviewText", "summary"]]
df_review = df_review.rename(columns={"asin": "product_id", "overall": "rating"})
print(len(df_review))

459436


In [4]:
### load the mega data

mega_data = []
with gzip.open('meta_Software.json.gz') as f:
    for l in f:
        mega_data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(mega_data))

# first row of the list
print(mega_data[1])

26790
{'category': [], 'tech1': '', 'description': ['', '<b>Latin rhythms that will get your kids singing in Spanish</b>', '<i>Sing, Watch, and Learn Spanish</i> helps your kids ages four through eight take a giant step in learning Spanish by combining two time-honored methods kids have always used to develop their language skills: Imitating other kids and singing along with simple melodies. This charming DVD contains 16 music videos featuring kids engaged in fun activities, from visiting animals at the zoo to comparing clothing sizes in grandmas closet.', 'Each video features an original song of authentic Latin rhythms that gets kids singing along with the children on screen. As they watch, listen, and sing along, kids absorb 300 Spanish words, each of which is shouted out in a song and displayed as a subtitle on screen.', '', '', ''], 'fit': '', 'title': 'Sing, Watch, &amp; Learn Spanish (DVD + Guide): 16 Fun Music Videos to Introduce Your Child to Spanish', 'also_buy': [], 'tech2': 

In [5]:
# convert list into pandas dataframe

df_product = pd.DataFrame.from_dict(mega_data)

#filter out the non-verified reviews, and leave only the columns we want
df_product = df_product[["asin", "rank"]]
df_product = df_product.rename(columns={"asin": "product_id"})
print(len(df_product))
df_product.head()
# TODO:filter out the products that has reviews, rank in software
# if(product_id)
# df_software = df_product[df_product['rank']]

26790


,product_id,rank
0,0030672120,"25,550 in Software ("
1,0071480935,"15,792 in Software ("
2,007329506X,"16,900 in Software ("
3,0073513458,"12,986 in Software ("
4,0073525758,"14,861 in Software ("


In [6]:
df_product = df_product.dropna(axis='index')
print(len(df_product))
df_product = df_product[df_product['rank'].str.contains('Software', na=False)]
print(len(df_product))


26790
21316


In [7]:

def toInt(str_in):
    try:
        str_in=''.join(i for i in str_in if i.isdigit())
        output = str_in
    except:
        output = np.nan
        
    return output


In [8]:
df_product['rank'] = df_product['rank'].apply(toInt)
df_product.head()

,product_id,rank
0,0030672120,25550
1,0071480935,15792
2,007329506X,16900
3,0073513458,12986
4,0073525758,14861


In [9]:
all_products_Y = {set}

for index, product_id, rank in df_product.itertuples():
    all_products_Y.add(product_id)
# print(all_products_Y)
print(len(all_products_Y))

17142


In [10]:
options = all_products_Y
  
# selecting rows that product ids are included in the product dataframe
df_review = df_review[df_review['product_id'].isin(options)]

In [11]:
np.random.seed(32) # for reproducible results.
df_product_x1 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[0]
df_product_x2 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[1]
df_product_x3 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[2]
df_product_x4 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[3]
df_product_x5 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[4]
df_product_x6 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[5]
df_product_x7 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[6]
df_product_x8 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[7]
df_product_x9 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[8]
#test set
df_product_y = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[9]
#training set
# df_product_x = df_product_x1
df_product_x = pd.concat([df_product_x1, df_product_x2, df_product_x3, df_product_x4, df_product_x5, df_product_x6, df_product_x7, df_product_x8, df_product_x9])
print(len(df_product), len(df_product_x), len(df_product_y), (len(df_product_x) + len(df_product_y)))
df_product.head()
del(df_product_x1, df_product_x2, df_product_x3, df_product_x4, df_product_x5, df_product_x6, df_product_x7, df_product_x8, df_product_x9)
# get = df_product_y.loc[df_product_y['product_id'] == '0077613252']
# get.head()

21316 19185 2131 21316


In [12]:
all_products_train = {set}
product_rank = {}
y = []

for index, product_id, rank in df_product_x.itertuples():
    all_products_train.add(str(product_id))   

def ranking(products):
    a = 0
    s = 0
    d = 0
    f = 0
    g = 0
    h = 0
    j = 0
    k = 0
    l = 0
    m = 0
    for index, product_id, rank in products.itertuples():
        if int(rank)<1000:
            product_rank.update({str(product_id): 1})
            a+=1
        elif int(rank)<5000:
            product_rank.update({str(product_id): 2})
            s+=1
        elif int(rank)<10000:
            product_rank.update({str(product_id): 3})
            d+=1
        elif int(rank)<15000:
            product_rank.update({str(product_id): 4})
            f+=1
        elif int(rank)<20000:
            product_rank.update({str(product_id): 5})
            g+=1
        elif int(rank)<27000:
            product_rank.update({str(product_id): 6})
            h+=1
        elif int(rank)<35000:
            product_rank.update({str(product_id): 7})
            l+=1
        elif int(rank)<45000:
            product_rank.update({str(product_id): 8})
            j+=1
        elif int(rank)<60000:
            product_rank.update({str(product_id): 9})
            m+=1
        else:
            product_rank.update({str(product_id):10})
            k+=1
    print(a,s,d,f,g,h,l,j,m,k)
            
ranking(df_product)
# print(all_products)
 
# print(product_rank['0030672120'])

281 1606 2456 2436 2184 2698 2162 2179 2524 2790


In [13]:
ratingsPerItem = defaultdict(list)
reviewsPerItem = defaultdict(list)
summariesPerItem = defaultdict(list)
ratingsPerItem_test = defaultdict(list)
reviewsPerItem_test = defaultdict(list)
summariesPerItem_test = defaultdict(list)

for index, rating, reviewTime, product_id, reviewText, summary in df_review.itertuples():
    if(product_id in all_products_train):
        ratingsPerItem[product_id].append(rating)
        reviewsPerItem[product_id].append(reviewText)
        summariesPerItem[product_id].append(summary)
    else:
        ratingsPerItem_test[product_id].append(rating)
        reviewsPerItem_test[product_id].append(reviewText)
        summariesPerItem_test[product_id].append(summary)
print(len(ratingsPerItem_test))
print(len(ratingsPerItem))
        
# print(len(all_products))
# print(len(df_review))
# df_review.head()
# print(ratingsPerItem['0077613252'])
# print(len(ratingsPerItem))
# # print(s)
# print(summariesPerItem['0077613252'])

5610
11530


In [14]:
# Xtrain = []
ytrain = []

def feature(datum):
#     Xtrain.append(datum[1])
    ytrain.append(int(product_rank[datum[0]]))
    feat = []
    feat.append(1)# offset
    feat.append(len(datum[1])) 
    rl = []
    sl = []
    for reviews in reviewsPerItem[datum[0]]:
        rl.append(len(str(reviews))) #length of reviews
    for summary in summariesPerItem[datum[0]]:
        sl.append(len(str(summary))) #length of summaries
    feat.append(sum(rl)/len(rl)) 
    feat.append(sum(sl)/len(sl))
    feat.append(sum(datum[1])/len(datum[1])) #average rating
#     feat.append()
    return feat

# Xtest = []
ytest = []
def feature_test(datum):
#     Xtest.append(datum[1])
    ytest.append(int(product_rank[(datum[0])]))
    feat = []
    feat.append(1)# offset
    feat.append(len(datum[1])) 
    rl = []
    sl = []
    for reviews in reviewsPerItem_test[datum[0]]:
        rl.append(len(str(reviews))) #length of reviews
    for summary in summariesPerItem_test[datum[0]]:
        sl.append(len(str(summary))) #length of summaries
    feat.append(sum(rl)/len(rl)) 
    feat.append(sum(sl)/len(sl))
    feat.append(sum(datum[1])/len(datum[1])) #average rating
#     feat.append()
    return feat

X_train = [feature(d) for d in ratingsPerItem.items()]
X_test = [feature_test(d) for d in ratingsPerItem_test.items()]

# Y = [feature(d) for d in ratingsPerItem.items()]
# print(product_rank['77775473'])
print(len(X_train), len(X_test), len(ytrain), len(ytest))

11530 5610 11530 5610


In [15]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [17]:
# mod = linear_model.LogisticRegression(solver = 'saga')
mod = linear_model.Ridge(1.0, fit_intercept=False) # MSE + 1.0 l2
mod.fit(X_train, ytrain) 

Ridge(fit_intercept=False)

In [18]:
print(len(X_train), len(ytrain))

11530 11530


In [19]:
pred = mod.predict(X_train)

In [20]:
#MSE on train set
MSE(pred, ytrain)

6.0863985571871995

In [21]:
pred = mod.predict(X_test)
#MSE on test set
MSE(pred, ytest)

6.061214209020762

In [24]:
#box of words
nltk.download('stopwords')
stopwords_set = (stopwords.words('english'))

wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for item in reviewsPerItem.items():
    for reviews in item[1]:
        review = ''.join([c for c in str(reviews).lower() if not c in punctuation])
    for w in review.split():
        if w not in stopwords_set and not w.isdigit():
            wordCount[w] += 1

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pecef\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:1000]]
# print(words)
# print(wordCount['trend'])
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [47]:
def feature_words(datum):
    feat = [0]*len(words)
    r = ''
    for d in datum[1]:
        r = r.join('').join([c for c in str(d).lower() if not c in punctuation])
    for w in r.split():
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) # offset
    return feat


X_train = [feature_words(d) for d in reviewsPerItem.items()]

In [49]:
mod = linear_model.Ridge(1.0, fit_intercept=False) # MSE + 1.0 l2
print(len(X_train), len(ytrain))
print(len(reviewsPerItem))
mod.fit(X_train, ytrain) 

11530 11530
11530


Ridge(fit_intercept=False)

In [50]:
pred = mod.predict(X_train)
MSE(pred, ytrain)

5.732992303912787

In [ ]:
#This MSE slightly better than then the recent